In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a simplified VGGNet
class SimplifiedVGGNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(SimplifiedVGGNet, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Output: 64x32x32
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: 128x16x16
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x4x4

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 512x2x2
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_prob),
            nn.Linear(512 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_prob),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimplifiedVGGNet(num_classes=10, dropout_prob=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(test_loader), correct / total

# Training loop
epochs = 20
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Print the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters in the simplified VGGNet: {count_parameters(model)}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 1.8259, Val Loss: 1.4919, Val Acc: 0.4237
Epoch 2/20, Train Loss: 1.4540, Val Loss: 1.3050, Val Acc: 0.5129
Epoch 3/20, Train Loss: 1.2453, Val Loss: 1.1294, Val Acc: 0.5919
Epoch 4/20, Train Loss: 1.1003, Val Loss: 1.0126, Val Acc: 0.6324
Epoch 5/20, Train Loss: 1.0057, Val Loss: 0.9109, Val Acc: 0.6772
Epoch 6/20, Train Loss: 0.9187, Val Loss: 0.8651, Val Acc: 0.7000
Epoch 7/20, Train Loss: 0.8637, Val Loss: 0.8479, Val Acc: 0.7136
Epoch 8/20, Train Loss: 0.8149, Val Loss: 0.7790, Val Acc: 0.7358
Epoch 9/20, Train Loss: 0.7622, Val Loss: 0.7286, Val Acc: 0.7530
Epoch 10/20, Train Loss: 0.7370, Val Loss: 0.7039, Val Acc: 0.7567
Epoch 11/20, Train Loss: 0.7129, Val Loss: 0.6947, Val Acc: 0.7655
Epoch 12/20, Train Loss: 0.6841, Val Loss: 0.6673, Val Acc: 0.7766
Epoch 13/20, Train Loss: 0.6625, Val Loss: 0.6696, Val Acc: 0.7726
Epoch 14/20, Train Loss: 0.6443, Val Loss: 0.6345, Val Acc: 0.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a simplified VGGNet
class SimplifiedVGGNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0):
        super(SimplifiedVGGNet, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Output: 64x32x32
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: 128x16x16
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x4x4

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 512x2x2
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_prob),
            nn.Linear(512 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_prob),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimplifiedVGGNet(num_classes=10, dropout_prob=0).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(test_loader), correct / total

# Training loop
epochs = 20
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Print the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters in the simplified VGGNet: {count_parameters(model)}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 1.8644, Val Loss: 1.5231, Val Acc: 0.4147
Epoch 2/20, Train Loss: 1.4135, Val Loss: 1.2670, Val Acc: 0.5373
Epoch 3/20, Train Loss: 1.1801, Val Loss: 1.0491, Val Acc: 0.6251
Epoch 4/20, Train Loss: 1.0041, Val Loss: 1.0210, Val Acc: 0.6394
Epoch 5/20, Train Loss: 0.8961, Val Loss: 0.8483, Val Acc: 0.7110
Epoch 6/20, Train Loss: 0.8144, Val Loss: 0.7976, Val Acc: 0.7256
Epoch 7/20, Train Loss: 0.7408, Val Loss: 0.7407, Val Acc: 0.7476
Epoch 8/20, Train Loss: 0.6985, Val Loss: 0.6725, Val Acc: 0.7697
Epoch 9/20, Train Loss: 0.6588, Val Loss: 0.7271, Val Acc: 0.7601
Epoch 10/20, Train Loss: 0.6241, Val Loss: 0.7123, Val Acc: 0.7631
Epoch 11/20, Train Loss: 0.6040, Val Loss: 0.6014, Val Acc: 0.7953
Epoch 12/20, Train Loss: 0.5766, Val Loss: 0.6341, Val Acc: 0.7879
Epoch 13/20, Train Loss: 0.5600, Val Loss: 0.6503, Val Acc: 0.7756
Epoch 14/20, Train Loss: 0.5443, Val Loss: 0.5938, Val Acc: 0.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a simplified VGGNet
class SimplifiedVGGNet(nn.Module):
    def __init__(self, num_classes=100, dropout_prob=0):
        super(SimplifiedVGGNet, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Output: 64x32x32
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: 128x16x16
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x4x4

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 512x2x2
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_prob),
            nn.Linear(512 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_prob),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimplifiedVGGNet(num_classes=100, dropout_prob=0).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(test_loader), correct / total

# Training loop
epochs = 20
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Print the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters in the simplified VGGNet: {count_parameters(model)}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 4.3372, Val Loss: 4.1800, Val Acc: 0.0509
Epoch 2/20, Train Loss: 3.8993, Val Loss: 3.7572, Val Acc: 0.1048
Epoch 3/20, Train Loss: 3.6536, Val Loss: 3.5621, Val Acc: 0.1385
Epoch 4/20, Train Loss: 3.4570, Val Loss: 3.3107, Val Acc: 0.1864
Epoch 5/20, Train Loss: 3.2695, Val Loss: 3.1809, Val Acc: 0.2130
Epoch 6/20, Train Loss: 3.0972, Val Loss: 3.0630, Val Acc: 0.2408
Epoch 7/20, Train Loss: 2.9506, Val Loss: 2.8835, Val Acc: 0.2790
Epoch 8/20, Train Loss: 2.8310, Val Loss: 2.7588, Val Acc: 0.3053
Epoch 9/20, Train Loss: 2.6965, Val Loss: 2.6650, Val Acc: 0.3195
Epoch 10/20, Train Loss: 2.5888, Val Loss: 2.6133, Val Acc: 0.3363
Epoch 11/20, Train Loss: 2.4982, Val Loss: 2.5416, Val Acc: 0.3437
Epoch 12/20, Train Loss: 2.4154, Val Loss: 2.4730, Val Acc: 0.3613
Epoch 13/20, Train Loss: 2.3414, Val Loss: 2.4366, Val Acc: 0.3712
Epoch 14/20, Train Loss: 2.2662, Val Loss: 2.3827, Val Acc: 0.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a simplified VGGNet
class SimplifiedVGGNet(nn.Module):
    def __init__(self, num_classes=100, dropout_prob=0.5):
        super(SimplifiedVGGNet, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Output: 64x32x32
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x16x16

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: 128x16x16
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x8x8

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 256x8x8
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x4x4

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # Output: 512x4x4
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 512x2x2
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_prob),
            nn.Linear(512 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_prob),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load CIFAR-10 dataset
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimplifiedVGGNet(num_classes=100, dropout_prob=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(test_loader), correct / total

# Training loop
epochs = 20
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Print the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of parameters in the simplified VGGNet: {count_parameters(model)}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Train Loss: 4.4051, Val Loss: 4.0772, Val Acc: 0.0482
Epoch 2/20, Train Loss: 4.0120, Val Loss: 3.9226, Val Acc: 0.0776
Epoch 3/20, Train Loss: 3.8382, Val Loss: 3.6990, Val Acc: 0.1112
Epoch 4/20, Train Loss: 3.6836, Val Loss: 3.5364, Val Acc: 0.1441
Epoch 5/20, Train Loss: 3.5373, Val Loss: 3.3510, Val Acc: 0.1746
Epoch 6/20, Train Loss: 3.4167, Val Loss: 3.2432, Val Acc: 0.1893
Epoch 7/20, Train Loss: 3.3180, Val Loss: 3.1654, Val Acc: 0.2190
Epoch 8/20, Train Loss: 3.2288, Val Loss: 3.0538, Val Acc: 0.2407
Epoch 9/20, Train Loss: 3.1515, Val Loss: 2.9780, Val Acc: 0.2539
Epoch 10/20, Train Loss: 3.0754, Val Loss: 2.9118, Val Acc: 0.2663
Epoch 11/20, Train Loss: 3.0057, Val Loss: 2.8379, Val Acc: 0.2766
Epoch 12/20, Train Loss: 2.9512, Val Loss: 2.7504, Val Acc: 0.2942
Epoch 13/20, Train Loss: 2.8926, Val Loss: 2.7475, Val Acc: 0.2974
Epoch 14/20, Train Loss: 2.8503, Val Loss: 2.7468, Val Acc: 0.